In [1]:
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from cvxopt import matrix 
from cvxopt import solvers
import time
from sklearn.metrics.pairwise import rbf_kernel, polynomial_kernel
from copy import copy
from sklearn.metrics import confusion_matrix

In [2]:
def load_mnist(path, kind='train'):
    import os
    import gzip
    import numpy as np

    labels_path = os.path.join(path,
                               '%s-labels-idx1-ubyte.gz'
                               % kind)
    images_path = os.path.join(path,
                               '%s-images-idx3-ubyte.gz'
                               % kind)

    with gzip.open(labels_path, 'rb') as lbpath:
        labels = np.frombuffer(lbpath.read(), dtype=np.uint8,
                               offset=8)

    with gzip.open(images_path, 'rb') as imgpath:
        images = np.frombuffer(imgpath.read(), dtype=np.uint8,
                               offset=16).reshape(len(labels), 784)

    return images, labels



X_all_labels, y_all_labels = load_mnist('Data', kind='train')


indexLabel2 = np.where((y_all_labels==2))
xLabel2 =  X_all_labels[indexLabel2][:1000,:].astype('float64') 
yLabel2 = y_all_labels[indexLabel2][:1000].astype('float64') 

indexLabel4 = np.where((y_all_labels==4))
xLabel4 =  X_all_labels[indexLabel4][:1000,:].astype('float64') 
yLabel4 = y_all_labels[indexLabel4][:1000].astype('float64') 

indexLabel6 = np.where((y_all_labels==6))
xLabel6 =  X_all_labels[indexLabel6][:100,:].astype('float64') 
yLabel6 = y_all_labels[indexLabel6][:100].astype('float64') 

yLabel2[:] = +1
yLabel4[:] = -1

X = np.concatenate([xLabel2, xLabel4])
y = np.concatenate([yLabel2, yLabel4])

X_train, X_test, y_train, y_test = train_test_split(X, y, stratify = y, test_size=0.2, random_state=1845787) 

scaler = MinMaxScaler()
X_train=scaler.fit_transform(X_train)
X_test=scaler.fit_transform(X_test)

In [57]:
class Svm_dcmp:
    
    def __init__(self, X, y, gamma, C, kernel):
        
        self.X = X
        self.y = y
        self.alpha = np.zeros((X.shape[0]))
        self.b = 0
        self.C = C
        self.gamma = gamma
        self.kernel = kernel
        self.grad = np.ones(X.shape[0])
        self.alpha = np.zeros(X.shape[0])
        
    def predict(self,X):
        
        if self.kernel == "gauss":
            z = (self.alpha*self.y) @ self.kernel_gauss(self.X, X) + self.b
        if self.kernel == "poly":
            z = (self.alpha*self.y) @ self.kernel_poly(self.X, X) + self.b
        a = np.sign(z)    
        return a
    
    def kernel_gauss(self, X1, X2):
        return rbf_kernel(X1,X2, gamma = self.gamma)
    
    def kernel_poly(self, X1, X2):
        return polynomial_kernel(X1,X2, gamma = self.gamma)

    def get_working_set(self):
        
        # box constraints
        y = self.y.ravel(); C = self.C; alpha = self.alpha
        R = np.where((alpha < 1e-5) & (y == +1) | (alpha > C-1e-5) & (y == -1) | (alpha > 1e-5) & (alpha < C-1e-5))[0]
        S = np.where((alpha < 1e-5) & (y == -1) | (alpha > C-1e-5) & (y == +1) | (alpha > 1e-5) & (alpha < C-1e-5))[0]
        
        # negative gradient divided by y
        if self.kernel == "gauss":
            K = self.kernel_gauss(self.X, self.X)
        if self.kernel == "poly":
            K = self.kernel_poly(self.X, self.X)
            
        Q = np.outer(y,y) * K 
        grad = alpha @ Q - 1
        grady = - grad*y
        
        # I and J definition
        grady_dict = {i:grady[i] for i in range(len(grady))}
        
        R_dict = dict((k, grady_dict[k]) for k in R)
        indexed_R = {k: v for k, v in sorted(R_dict.items(), key=lambda item: item[1])}
        i = list(indexed_R.keys())[-1]
        
        S_dict = dict((k, grady_dict[k]) for k in S)
        indexed_S = {k: v for k, v in sorted(S_dict.items(), key=lambda item: item[1])}
        j = list(indexed_S.keys())[0]
        
        # optimality condition
        m = max(grady[R])
        M = min(grady[S])

        W = [i,j]
    
        d1 = y[i]
        d2 = -y[j]
        
        
        print(d1, d2)
        flag = False
        if m-M < 0.005:
            flag = True
            self.diff = m-M
        
        
        return W, grad, d1,d2, flag, Q[np.ix_(W, W)]
    
    def find_beta_max(self, d1,d2, alpha):
        
        beta_bar = 0
        
        if d1 > 0:
            if d2 > 0:
                beta_bar = min(self.C-alpha[0],self.C-alpha[1])
            else:
                beta_bar = min(self.C-alpha[0], alpha[1])
        else:
            if d2 > 0:
                beta_bar = min(alpha[0], self.C-alpha[1])
            else:
                beta_bar = min(alpha[0], alpha[1])
        
        return beta_bar
        
    def optimize(self):
        
        alpha_bar = np.zeros(2)
        
        for i in range(1000):
            W, grad, d1,d2, flag, Q = self.get_working_set()
            
            if flag:
                print("optimality reached")
                break
            
            beta_star = 0
            d = np.array([d1,d2]).reshape(-1,1)
            d_star = np.zeros(2)

            if grad[W] @ d == 0:
                pass
            else:
                if grad[W] @ d < 0:
                    d_star = d
                else:
                    d_star = -d

                beta_bar = self.find_beta_max(d_star[0],d_star[1], alpha_bar)

                if beta_bar == 0:
                    beta_star = 0

                elif d_star.T @ Q @ d_star == 0:
                    beta_star == beta_bar

                else:
                    if d_star.T @ Q @ d_star > 0:
                        beta_nv = (-grad[W] @ d_star)/(d_star.T @ Q @ d_star)
                        beta_star = min(beta_bar, beta_nv)

            alpha_star = alpha_bar + beta_star * d_star.T
            self.alpha[W] = alpha_star
            
    
svm = Svm_dcmp(X_train, y_train, gamma = 0.01, C = 2, kernel = "gauss")
svm.optimize()


1.0 1.0
[[1.68365058]] 2.0
1.0 1.0
[[2.16172654]] 2.0
1.0 1.0
[[1.94571194]] 2.0
1.0 1.0
[[2.06820396]] 2.0
1.0 1.0
[[1.8587705]] 2.0
1.0 1.0
[[2.09564159]] 2.0
1.0 1.0
[[2.03079328]] 2.0
1.0 1.0
[[1.9153285]] 2.0
1.0 1.0
[[2.07876139]] 2.0
1.0 1.0
[[1.94373476]] 2.0
1.0 1.0
[[2.27808637]] 2.0
1.0 1.0
[[1.80343956]] 2.0
1.0 1.0
[[2.13955065]] 2.0
1.0 1.0
[[2.46860787]] 2.0
1.0 1.0
[[1.88054338]] 2.0
1.0 1.0
[[2.02601659]] 2.0
1.0 1.0
[[1.90389723]] 2.0
1.0 1.0
[[1.95397142]] 2.0
1.0 1.0
[[2.04557907]] 2.0
1.0 1.0
[[2.03847169]] 2.0
1.0 1.0
[[2.09982666]] 2.0
1.0 1.0
[[2.19930759]] 2.0
1.0 1.0
[[1.69527052]] 2.0
1.0 1.0
[[1.71698144]] 2.0
1.0 1.0
[[2.05280035]] 2.0
1.0 1.0
[[1.93489933]] 2.0
1.0 1.0
[[1.8079132]] 2.0
1.0 1.0
[[1.74469893]] 2.0
1.0 1.0
[[2.3175304]] 2.0
1.0 1.0
[[2.09743416]] 2.0
1.0 1.0
[[2.16088538]] 2.0
1.0 1.0
[[1.7514379]] 2.0
1.0 1.0
[[2.09222183]] 2.0
1.0 1.0
[[1.98231679]] 2.0
1.0 1.0
[[1.71403261]] 2.0
1.0 1.0
[[1.93365741]] 2.0
1.0 1.0
[[2.16510769]] 2.0
1.0 1.

KeyboardInterrupt: 